In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential,load_model,save_model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D
from keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam

In [2]:
train_datagen = ImageDataGenerator(rescale = 1./255,
      rotation_range=25,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [3]:
batch_size = 64
target_size = (64, 64)
input_shape=(64, 64, 3)
seed=1337
adam = 0.001
fre= -20
FC = 2048
E = 1
patience = 3
verbose = 1
factor = 0.50
min_lr = 0.0001
steps_per_epoch=256
validation_steps=256
epochs=8

In [4]:
test_datagen = ImageDataGenerator( rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory('I:\study\dpl\project\Dataset\Train',
                                                    batch_size =batch_size ,
                                                    class_mode = 'binary',
                                                    seed=seed,
                                                    target_size = target_size )

validation_generator =  test_datagen.flow_from_directory( 'I:\study\dpl\project\Dataset\Validation',
                                                          batch_size  = batch_size,
                                                          class_mode  = 'binary',
                                                          seed=seed,
                                                          target_size = target_size)

Found 160000 images belonging to 2 classes.
Found 22598 images belonging to 2 classes.


In [5]:
base_model = tf.keras.applications.VGG16(input_shape=input_shape,include_top=False,weights="imagenet")

In [6]:
model=Sequential()
model.add(base_model)
model.add(layers.Dropout(.2))


# Add new layers
model.add(Flatten())
model.add(Dense(FC , activation='relu'))
model.add(layers.Dropout(.2))
model.add(Dense(FC , activation='relu'))
model.add(layers.Dropout(.2))
model.add(Dense(FC, activation='relu'))
model.add(layers.Dropout(.2))
model.add(Dense(E, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 dropout (Dropout)           (None, 2, 2, 512)         0         
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 2048)              4196352   
                                                                 
 dropout_1 (Dropout)         (None, 2048)              0         
                                                                 
 dense_1 (Dense)             (None, 2048)              4196352   
                                                                 
 dropout_2 (Dropout)         (None, 2048)              0

In [7]:
model.compile(optimizer=Adam(adam),
              loss='binary_crossentropy'
              ,metrics=['accuracy'])

In [8]:
lrd = ReduceLROnPlateau(monitor = 'val_loss',
                        patience = patience,
                        verbose = verbose ,
                        factor = factor,
                        min_lr = min_lr)

mcp = ModelCheckpoint('model.h5')

es = EarlyStopping(verbose=verbose, patience=patience)

In [ ]:
import time
start = time.time()
hist = model.fit_generator(generator=train_generator,
                           validation_data=validation_generator,
                           steps_per_epoch=steps_per_epoch,
                           validation_steps=validation_steps,
                           epochs=epochs,
                           callbacks=[lrd, mcp, es])
end = time.time()
print(end - start)

C:\Users\asus\AppData\Local\Temp\ipykernel_19300\1842380108.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(generator=train_generator,


Epoch 1/8
256/256 [==============================] - 398s 2s/step - loss: 1.1551 - accuracy: 0.5659 - val_loss: 0.6740 - val_accuracy: 0.6143 - lr: 0.0010
Epoch 2/8
256/256 [==============================] - 288s 1s/step - loss: 0.6808 - accuracy: 0.5796 - val_loss: 0.6725 - val_accuracy: 0.6066 - lr: 0.0010
Epoch 3/8
256/256 [==============================] - 250s 970ms/step - loss: 0.6815 - accuracy: 0.5789 - val_loss: 0.6686 - val_accuracy: 0.6129 - lr: 0.0010
Epoch 4/8
256/256 [==============================] - 240s 932ms/step - loss: 0.6796 - accuracy: 0.5829 - val_loss: 0.6707 - val_accuracy: 0.6107 - lr: 0.0010
Epoch 5/8
256/256 [==============================] - 254s 984ms/step - loss: 0.6790 - accuracy: 0.5850 - val_loss: 0.6702 - val_accuracy: 0.6116 - lr: 0.0010
Epoch 6/8
120/256 [=============>................] - ETA: 1:45 - loss: 0.6820 - accuracy: 0.5754

In [ ]:
import matplotlib.pyplot as plt
acc = hist.history['accuracy']
val_acc = hist.history['val_accuracy']
loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'g', label='Training accuracy')
plt.plot(epochs, val_acc, 'y', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
test_generator =  test_datagen.flow_from_directory( 'I:\study\dpl\project\Dataset\Test',
                                                          batch_size  = batch_size,
                                                          class_mode  = 'binary',
                                                          seed=seed,
                                                          target_size = target_size)
score = model.evaluate_generator(generator=validation_generator,steps=256)

In [ ]:
print(score)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras.utils as image
#  images test 1
path_testmodel = "I:/study/dpl/project/Dataset/testmodel/test1.jpg"
imge = image.load_img(path_testmodel, target_size=target_size)

X = image.img_to_array(imge)
X = np.expand_dims(X, axis=0)


images = np.vstack([X])
classes = model.predict(images, batch_size=1)
print(classes[0])
if classes[0]>0.5:
    print("This is a male")
else:
    print( "This  is a female")
plt.imshow(imge)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras.utils as image
#  images test 2
path_testmodel = "I:/study/dpl/project/Dataset/testmodel/test2.jpg"
imge = image.load_img(path_testmodel, target_size=target_size)
X = image.img_to_array(imge)
X = np.expand_dims(X, axis=0)

images = np.vstack([X])
classes = model.predict(images, batch_size=1)
print(classes[0])
if classes[0]>0.5:
    print("This is a male")
else:
    print( "This is a female")

plt.imshow(imge)